Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [92]:
# Graph
from math import sqrt
batch_size = 128
H = 1024
beta = 1e-4

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels], stddev=sqrt(3.0/((image_size * image_size )+ num_labels))))
    biases1 = tf.Variable(tf.zeros([num_labels]))

    weights2 = tf.Variable(
        tf.truncated_normal([num_labels,H], stddev=sqrt(3.0/(H + num_labels))))
    
    weights3 = tf.Variable(
        tf.truncated_normal([H, num_labels], stddev=sqrt(3.0/(H + num_labels))))
    biases3 = tf.Variable(tf.zeros([num_labels]))
    
  
    # Training computation
    a1 = tf.matmul(tf_train_dataset, weights1) + biases1
    a2 = tf.nn.relu(tf.matmul(a1, weights2))
    #a2 = tf.tanh(tf.matmul(a1, weights2))
    logits = tf.matmul(a2, weights3) + biases3

    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) +
                          beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + 
                                  tf.nn.l2_loss(weights3)))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_a1 = tf.matmul(tf_valid_dataset, weights1) + biases1
    valid_a2 = tf.nn.relu(tf.matmul(valid_a1, weights2))
    #valid_a2 = tf.tanh(tf.matmul(valid_a1, weights2))
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_a2, weights3) + biases3)
    
    test_a1 = tf.matmul(tf_test_dataset, weights1) + biases1
    test_a2 = tf.nn.relu(tf.matmul(test_a1, weights2))
    #test_a2 = tf.tanh(tf.matmul(test_a1, weights2))
    test_prediction = tf.nn.softmax(
        tf.matmul(test_a2, weights3) + biases3)

In [93]:
# Run the Graph
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.332665
Minibatch accuracy: 6.2%
Validation accuracy: 30.2%
Minibatch loss at step 500: 0.826725
Minibatch accuracy: 74.2%
Validation accuracy: 83.1%
Minibatch loss at step 1000: 0.437080
Minibatch accuracy: 88.3%
Validation accuracy: 83.9%
Minibatch loss at step 1500: 0.605762
Minibatch accuracy: 84.4%
Validation accuracy: 84.5%
Minibatch loss at step 2000: 0.374016
Minibatch accuracy: 87.5%
Validation accuracy: 84.2%
Minibatch loss at step 2500: 0.600439
Minibatch accuracy: 82.8%
Validation accuracy: 84.3%
Minibatch loss at step 3000: 0.527833
Minibatch accuracy: 87.5%
Validation accuracy: 84.3%
Minibatch loss at step 3500: 0.405960
Minibatch accuracy: 87.5%
Validation accuracy: 84.7%
Minibatch loss at step 4000: 0.625970
Minibatch accuracy: 82.8%
Validation accuracy: 84.4%
Minibatch loss at step 4500: 0.345050
Minibatch accuracy: 89.8%
Validation accuracy: 85.2%
Minibatch loss at step 5000: 0.472144
Minibatch accuracy: 85.9%
Validation accuracy

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [98]:
# Run the Graph
num_steps = 200

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.316006
Minibatch accuracy: 6.2%
Validation accuracy: 24.4%
Test accuracy: 88.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [103]:
# Graph
from math import sqrt
batch_size = 128
H = 1024
beta = 1e-4

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels], stddev=sqrt(3.0/((image_size * image_size )+ num_labels))))
    biases1 = tf.Variable(tf.zeros([num_labels]))

    weights2 = tf.Variable(
        tf.truncated_normal([num_labels,H], stddev=sqrt(3.0/(H + num_labels))))
    
    weights3 = tf.Variable(
        tf.truncated_normal([H, num_labels], stddev=sqrt(3.0/(H + num_labels))))
    biases3 = tf.Variable(tf.zeros([num_labels]))

    tf_keep_prob = tf.constant(0.5)
    # Training computation
    a1 = tf.matmul(tf_train_dataset, weights1) + biases1
    a2 = tf.nn.dropout(tf.nn.relu(tf.matmul(a1, weights2)),tf_keep_prob)
    logits = tf.matmul(a2, weights3) + biases3
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) +
                          beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + 
                                  tf.nn.l2_loss(weights3)))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_a1 = tf.matmul(tf_valid_dataset, weights1) + biases1
    valid_a2 = tf.nn.relu(tf.matmul(valid_a1, weights2))
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_a2, weights3) + biases3)
    
    test_a1 = tf.matmul(tf_test_dataset, weights1) + biases1
    test_a2 = tf.nn.relu(tf.matmul(test_a1, weights2))
    test_prediction = tf.nn.softmax(
        tf.matmul(test_a2, weights3) + biases3)

In [104]:
# Run the Graph
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.347893
Minibatch accuracy: 8.6%
Validation accuracy: 17.2%
Minibatch loss at step 500: 0.924175
Minibatch accuracy: 72.7%
Validation accuracy: 83.0%
Minibatch loss at step 1000: 0.499427
Minibatch accuracy: 84.4%
Validation accuracy: 82.8%
Minibatch loss at step 1500: 0.721231
Minibatch accuracy: 81.2%
Validation accuracy: 84.5%
Minibatch loss at step 2000: 0.486349
Minibatch accuracy: 85.9%
Validation accuracy: 83.9%
Minibatch loss at step 2500: 0.675021
Minibatch accuracy: 79.7%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 0.605935
Minibatch accuracy: 82.8%
Validation accuracy: 82.2%
Test accuracy: 88.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [135]:
# Graph
from math import sqrt
batch_size = 128
H1 = 2048
H2 = 1024
H3 = 1024
beta = 1e-4

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    #Layer 1
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, H1], stddev=sqrt(3.0/((image_size * image_size )+ H1))))
    biases1 = tf.Variable(tf.zeros([H1]))
    activation_weights1 = tf.Variable(
        tf.truncated_normal([H1,H1], stddev=sqrt(3.0/(H + num_labels))))
    
    #Layer 2
    weights2 = tf.Variable(
        tf.truncated_normal([H1, H2], stddev=sqrt(3.0/(H1 + H2))))
    biases2 = tf.Variable(tf.zeros([H2]))
    activation_weights2 = tf.Variable(
        tf.truncated_normal([H2,H2], stddev=sqrt(3.0/(H + num_labels))))

    #Layer 3
    weights3 = tf.Variable(
        tf.truncated_normal([H2, H3], stddev=sqrt(3.0/(H2 + H3))))
    biases3 = tf.Variable(tf.zeros([H3]))
    activation_weights3 = tf.Variable(
        tf.truncated_normal([H3,H3], stddev=sqrt(3.0/(H + num_labels))))

    #Output Layer
    weights4 = tf.Variable(
        tf.truncated_normal([H3, num_labels], stddev=sqrt(3.0/(H3 + num_labels))))
    biases4 = tf.Variable(tf.zeros([num_labels]))




    tf_keep_prob = tf.constant(0.5)
    # Training computation
    layer1 = tf.matmul(tf_train_dataset, weights1) + biases1
    a1 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer1, activation_weights1)),tf_keep_prob)
    
    layer2 = tf.matmul(a1, weights2) + biases2
    a2 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer2, activation_weights2)),tf_keep_prob)
    
    layer3 = tf.matmul(a2, weights3) + biases3
    a3 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer3, activation_weights3)),tf_keep_prob)
    
    logits = tf.matmul(a3, weights4) + biases4
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) +
                          beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + 
                                  tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4) +
                                  tf.nn.l2_loss(activation_weights1) + 
                                  tf.nn.l2_loss(activation_weights2) + 
                                  tf.nn.l2_loss(activation_weights3)))
  
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    starter_learning_rate = 0.1
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
     
    valid_layer1 = tf.matmul(tf_valid_dataset, weights1) + biases1
    valid_a1 = tf.nn.relu(tf.matmul(valid_layer1, activation_weights1))
    valid_layer2 = tf.matmul(valid_a1, weights2) + biases2
    valid_a2 = tf.nn.relu(tf.matmul(valid_layer2, activation_weights2))
    valid_layer3 = tf.matmul(valid_a2, weights3) + biases3
    valid_a3 = tf.nn.relu(tf.matmul(valid_layer3, activation_weights3))
    valid_prediction = tf.nn.softmax(tf.matmul(valid_a3, weights4) + biases4)
    
    test_layer1 = tf.matmul(tf_test_dataset, weights1) + biases1
    test_a1 = tf.nn.relu(tf.matmul(test_layer1, activation_weights1))
    test_layer2 = tf.matmul(test_a1, weights2) + biases2
    test_a2 = tf.nn.relu(tf.matmul(test_layer2, activation_weights2))
    test_layer3 = tf.matmul(test_a2, weights3) + biases3
    test_a3 = tf.nn.relu(tf.matmul(test_layer3, activation_weights3))
    test_prediction = tf.nn.softmax(tf.matmul(test_a3, weights4) + biases4)
    
# Run the Graph
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Learning Rate:", learning_rate.eval())
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.429143
Minibatch accuracy: 12.5%
Validation accuracy: 17.2%
Learning Rate: 0.1
Minibatch loss at step 1000: 0.953548
Minibatch accuracy: 86.7%
Validation accuracy: 86.0%
Learning Rate: 0.09216
Minibatch loss at step 2000: 0.918556
Minibatch accuracy: 89.8%
Validation accuracy: 87.3%
Learning Rate: 0.0849346
Minibatch loss at step 3000: 0.917707
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Learning Rate: 0.0782758
Minibatch loss at step 4000: 0.939141
Minibatch accuracy: 86.7%
Validation accuracy: 88.4%
Learning Rate: 0.072139
Minibatch loss at step 5000: 0.867144
Minibatch accuracy: 89.8%
Validation accuracy: 89.2%
Learning Rate: 0.0664833
Minibatch loss at step 6000: 0.952346
Minibatch accuracy: 85.9%
Validation accuracy: 89.2%
Learning Rate: 0.061271
Minibatch loss at step 7000: 0.823030
Minibatch accuracy: 89.8%
Validation accuracy: 89.5%
Learning Rate: 0.0564673
Minibatch loss at step 8000: 0.755896
Minibatch accuracy: 91.4%
Validatio